# 🤖 AgentPro Hiring Assistant

This notebook demonstrates how to build a Hiring Assistant using AgentPro that analyzes candidate profiles, resumes, and GitHub activity to provide detailed assessments for HR professionals.

### Features:
- Resume analysis from Google Drive URLs
- GitHub profile analysis
- Skills and experience matching
- Detailed candidate persona generation
- Company culture fit assessment


## Step 1: Clone AgentPro and Install Dependencies

To get started with **AgentPro**, begin by cloning the official GitHub repository and installing its dependencies.

In [ ]:
# !git clone https://github.com/traversaal-ai/AgentPro.git
%cd AgentPro
%pip install -r requirements.txt
%pip install gradio PyPDF2 python-docx beautifulsoup4 requests python-dotenv

## Step 2: Set Up API Keys

In [10]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
os.environ["TRAVERSAAL_ARES_API_KEY"] = "YOUR_TRAVERSAAL_API_KEY"

## Step 3: Create the Hiring Agent

In [11]:
from agentpro import AgentPro
from agentpro.tools import AresInternetTool, CodeEngine
import requests
from bs4 import BeautifulSoup
import PyPDF2
import io
import docx
from typing import Dict, List, Optional

class HiringAgent:
    def __init__(self):
        self.agent = AgentPro(tools=[AresInternetTool(), CodeEngine()])
        
    def extract_text_from_pdf(self, pdf_url: str) -> str:
        """Extract text from PDF file."""
        response = requests.get(pdf_url)
        pdf_file = io.BytesIO(response.content)
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text
    
    def extract_text_from_docx(self, docx_url: str) -> str:
        """Extract text from DOCX file."""
        response = requests.get(docx_url)
        docx_file = io.BytesIO(response.content)
        doc = docx.Document(docx_file)
        text = ""
        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"
        return text
    
    def analyze_github_profile(self, github_url: str) -> Dict:
        """Analyze GitHub profile and extract relevant information."""
        response = requests.get(github_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract basic information
        name = soup.find('span', {'class': 'p-name'}).text.strip() if soup.find('span', {'class': 'p-name'}) else ""
        bio = soup.find('div', {'class': 'p-note'}).text.strip() if soup.find('div', {'class': 'p-note'}) else ""
        
        # Extract repositories
        repos = []
        for repo in soup.find_all('a', {'data-hovercard-type': 'repository'})[:5]:
            repos.append({
                'name': repo.text.strip(),
                'url': f"https://github.com{repo['href']}"
            })
        
        return {
            'name': name,
            'bio': bio,
            'repositories': repos
        }
    
    def analyze_candidate(self, resume_url: str, github_url: str, job_description: str, company_info: str) -> Dict:
        """Analyze candidate profile and generate assessment."""
        # Extract resume text
        if resume_url.endswith('.pdf'):
            resume_text = self.extract_text_from_pdf(resume_url)
        elif resume_url.endswith('.docx'):
            resume_text = self.extract_text_from_docx(resume_url)
        else:
            resume_text = ""
        
        # Analyze GitHub profile
        github_data = self.analyze_github_profile(github_url)
        
        # Generate assessment using AgentPro
        prompt = f"""
        Analyze this candidate profile and provide a detailed assessment:
        
        Resume Content:
        {resume_text}
        
        GitHub Profile:
        Name: {github_data['name']}
        Bio: {github_data['bio']}
        Top Repositories: {[repo['name'] for repo in github_data['repositories']]}
        
        Job Description:
        {job_description}
        
        Company Information:
        {company_info}
        
        Please provide:
        1. Skills and experience match with job requirements
        2. Technical proficiency assessment
        3. Cultural fit analysis
        4. Strengths and areas for development
        5. Overall recommendation
        """
        
        assessment = self.agent(prompt)
        
        return {
            'resume_analysis': resume_text,
            'github_analysis': github_data,
            'assessment': assessment
        }

## Step 4: Create Simple Gradio Interface

In [ ]:
import gradio as gr

# Initialize the hiring agent
hiring_agent = HiringAgent()

def analyze_candidate(resume_url, github_url, job_description, company_info):
    try:
        result = hiring_agent.analyze_candidate(resume_url, github_url, job_description, company_info)
        return result['assessment']
    except Exception as e:
        return f"Error analyzing candidate: {str(e)}"

# Create a simple Gradio Interface
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("""
    # 🤖 AgentPro Hiring Assistant
    Upload candidate details and get a detailed assessment for your hiring process.
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📝 Candidate Information")
            resume_url = gr.Textbox(
                label="Resume URL",
                placeholder="Enter Google Drive URL of the resume",
                info="Paste the Google Drive URL of the candidate's resume"
            )
            github_url = gr.Textbox(
                label="GitHub Profile",
                placeholder="Enter GitHub profile URL",
                info="Paste the candidate's GitHub profile URL"
            )
            
            gr.Markdown("### 🏢 Job & Company Details")
            job_description = gr.Textbox(
                label="Job Description",
                placeholder="Enter the job description",
                lines=5,
                info="Describe the role and requirements"
            )
            company_info = gr.Textbox(
                label="Company Information",
                placeholder="Enter company details and culture",
                lines=3,
                info="Describe the company culture and environment"
            )
            
            analyze_btn = gr.Button("Analyze Candidate", variant="primary")
        
        with gr.Column(scale=2):
            gr.Markdown("### 📊 Assessment Results")
            output = gr.Markdown
    
    # Add example inputs
    gr.Markdown("### 💡 Example Inputs")
    gr.Examples(
        examples=[
            [
                "https://drive.google.com/example-resume.pdf",
                "https://github.com/example-user",
                "Looking for a Senior Python Developer with 5+ years of experience in web development, machine learning, and cloud technologies. Must have strong problem-solving skills and experience with agile methodologies.",
                "Tech startup focused on AI solutions, fast-paced environment, collaborative culture, emphasis on innovation and continuous learning."
            ]
        ],
        inputs=[resume_url, github_url, job_description, company_info],
        outputs=[output],
        fn=analyze_candidate,
        cache_examples=True
    )
    
    # Add footer
    gr.Markdown("""
    ---
    *Powered by AgentPro - An AI-powered hiring assistant*
    """)
    
    # Connect the analyze button
    analyze_btn.click(
        fn=analyze_candidate,
        inputs=[resume_url, github_url, job_description, company_info],
        outputs=[output]
    )

# Launch the app
app.launch(share=True)